In [76]:
import slate3k as slate
import pandas as pd
import re
import numpy as np
from tika import parser

##### ```tika``` uses a java pipeline and may require access to firewall, but it's the best parser I've found.

In [2]:
raw = parser.from_file('Films_Illustrating_Character_Strengths_and_Virtues.pdf')

2021-04-08 08:36:34,879 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


with open('Films_Illustrating_Character_Strengths_and_Virtues.pdf', 'rb') as f:
    extracted_text = slate.PDF(f)

In [3]:
header1 = "Positive Psychology at the Movies"
header2 = "Appendices"
header3 = "From: Niemiec, R.M., & Wedding, D. : Using Films to Build Virtues and Character Strengths. © 2008 Hogrefe & Huber Publishers (http://www.hogrefe.com/)"

char_strengths = ['WISDOM: Creativity (originality, ingenuity)',
                  ' WISDOM: Curiosity  (interest, novelty-seeking,  openness to experience)',
                  ' WISDOM: Open-mindedness (judgment, critical thinking)',
                  ' WISDOM: Love of Learning',
                  '  WISDOM: Perspective  (wisdom)',
                  ' COURAGE: Bravery (valor)',
                  ' COURAGE: Integrity  (authenticity, honesty)',
                  ' COURAGE: Vitality  (zest, enthusiasm, vigor, energy)',
                  ' COURAGE: Persistence  (perseverance, industriousness)',
                  ' HUMANITY: Love',
                  'HUMANITY: Kindness  (generosity, nurturance, care, compassion, altruistic love,  niceness)',
                  ' HUMANITY: Social  Intelligence  (emotional, personal intelligence)',
                  ' JUSTICE: Citizenship  (social responsibility, loyalty, teamwork)',
                  ' JUSTICE: Fairness',
                  'JUSTICE: Leadership ',
                  ' TEMPERANCE: Forgiveness and Mercy',
                  ' TEMPERANCE: Humility and Modesty',
                  ' TEMPERANCE: Prudence',
                  ' TEMPERANCE: Self-Regulation  (self-control)',
                  ' TRANSCENDENCE:  Appreciation of Beauty and Excellence  (awe, wonder, elevation)',
                  ' TRANSCENDENCE: Grati-tude',
                  ' TRANSCENDENCE: Hope  (optimism, future mindedness, future orientation)',
                  ' TRANSCENDENCE: Humor (playfulness)',
                  '   TRANSCENDENCE:  Spirituality  (religiousness, faith, purpose, meaning)'
                  ]

In [4]:
def find(string, substring):
    return [m.start() for m in re.finditer(substring, string)]

In [5]:
content = raw['content']
content = content[content.find("WISDOM"):]
content = content[:content.find("View publication")]
content = content.replace("\n", "")

content = content.replace(header1, "")
content = content.replace(header2, "")
content = content.replace(header3, "")
content = content.strip()

In [6]:
lines = []

all_indices = find(content, 'Ψ ')

for i, index in enumerate(all_indices):
    
    if i == 0:
        lines.append(content[0:index+1])
    else:
        lines.append(content[all_indices[i-1]+2:index+1])

In [139]:
csv_dict = {}
count_score = []
char_found = -1

for line in lines:
     
    previous_char = char_found
  
    for strength in char_strengths:
        # if any char strength is found in the line
        if line.find(strength.strip()) != -1:
            current_strength = strength.strip()
            char_found += 1

    if char_found != previous_char:
    
        clear_line = line.replace(current_strength, "").replace('Ψ', "").strip()
        
        # if there is a movie left
        if clear_line != "":
            csv_dict[clear_line] = current_strength.strip()
            count_score.append([float(line.count('Ψ'))])

    else:

        # if the movie already has a char strength:
        if line.replace('Ψ', "").strip() in list(csv_dict.keys()):
            count_score[list(csv_dict.keys()).index(line.replace('Ψ', "").strip())].append(float(line.count('Ψ')))
            csv_dict[line.replace('Ψ', "").strip()] += ';' + current_strength.strip()
        else:
            count_score.append([float(line.count('Ψ'))])
            csv_dict[line.replace('Ψ', "").strip()] = current_strength.strip()

In [147]:
dataframe_dict = {}
df_columns = [i.strip() for i in char_strengths]

for i, key in enumerate(csv_dict.keys()):

    binary_list = []
    
    binary_list.append(np.array(count_score[i]).mean())
    
    for strength in df_columns:
        if strength in csv_dict[key].split(';'):
            binary_list.append('1')
        else:
            binary_list.append('0')
        
    dataframe_dict[key.replace('Ψ', "").strip()] = binary_list

In [149]:
movie_df = pd.DataFrame.from_dict(dataframe_dict, orient='index', columns=df_columns.insert(0, 'Meaningfulness'))

In [152]:
renames = ['Meaningfulness',
           'Creativity (Wisdom)',
           'Curiosity (Wisdom)',
           'Open-mindedness (Wisdom)',
           'Love of learning (Wisdom)',
           'Perspective (Wisdom)',
           'Bravery (Courage)',
           'Integrity (Courage)',
           'Vitality (Courage)',
           'Persistence (Courage)',
           'Love (Humanity)',
           'Kindness (Humanity)',
           'Social Intelligence (Humanity)',
           'Citizenship (Justice)',
           'Fairness (Justice)',
           'Leadership (Justice)',
           'Forgiveness and mercy (Temperance)',
           'Humility and modesty (Temperance)',
           'Prudence (Temperance)',
           'Self-regulation (Temperance)',
           'Appreciation of Beauty and Excellence (Transcendence)',
           'Gratitude (Transcendence)',
           'Hope (Transcendence)',
           'Humor (Transcendence)',
           'Spirituality (Transcendence)'
           ]

renaming_dict = {}

for i, col in enumerate(list(movie_df.columns)):
    renaming_dict[col] = renames[i]

movie_df.rename(columns = renaming_dict, inplace = True) 

movie_df['movie_id'] = range(0, len(movie_df))

movie_df.to_csv('paper_dataset.csv')